In [ ]:
#@title Imports, load sequences, helper functions
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.utils import resample
# import matplotlib.pyplot as plt
# import matplotlib.lines as mlines
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, normalize
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import resample
# import matplotlib.pyplot as plt
# import matplotlib.lines as mlines
# import matplotlib.ticker as ticker
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
# from datasets import Dataset
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('./drive/MyDrive/LazBF_sequences.csv')
LazBF_sequences = df['sequences'].tolist()
LazBF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/LazBF_sample.csv')
LazBF_sample = df['sequences'].tolist()
LazBF_sample_labels = np.array(df['labels'].values)

df = pd.read_csv('./drive/MyDrive/LazDEF_sequences.csv')
LazDEF_sequences = df['sequences'].tolist()
LazDEF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/LazDEF_sample.csv')
LazDEF_sample = df['sequences'].tolist()
LazDEF_sample_labels = np.array(df['labels'].values)

lazbf_mlm_none = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_none.npy")
lazdef_mlm_none = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_none.npy")

lazbf_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazBF.npy")
lazdef_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazBF.npy")

lazbf_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazDEF.npy")
lazdef_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazDEF.npy")

metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Auroc']

def print_metrics(y_true, y_pred):
  metrics = [accuracy_score(y_true, y_pred), precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred), roc_auc_score(y_true, y_pred)]
  for i in range(len(metrics)):
    if metrics[i] < 0.5:
      metrics[i] = 1-metrics[i]
  # for met, name in zip(metrics, metric_names):
  #  print(f'{name}: {met}')
  return metrics

def print_avg_performance(performances):
  performances = np.array(performances)
  performances_mean = np.mean(performances, axis=0)
  performances_std = np.std(performances, axis=0)
  for met, std, name in zip(performances_mean, performances_std, metric_names):
    print(f'Avgerage {name}: {met} +- {std}')

In [ ]:
lazbf_mlm_PA = np.load("./drive/MyDrive/LazBF_mlm_PA.npy")
lazdef_mlm_PA = np.load("./drive/MyDrive/LazDEF_mlm_PA.npy")

In [ ]:
print("LazBF Peptide-ESM Embeddings")
performance_list = []
for i in range(5):
  kmeans = KMeans(n_clusters=2, random_state=i+1, n_init='auto')
  kmeans.fit(lazbf_mlm_PA)
  LazBF_pred = kmeans.labels_
  centroids = kmeans.cluster_centers_
  performance_list.append(print_metrics(LazBF_sample_labels, LazBF_pred))
print_avg_performance(performance_list)

In [ ]:
print("LazDEF Peptide-ESM Embeddings")
performance_list = []
for i in range(5):
  kmeans = KMeans(n_clusters=2, random_state=i+1, n_init='auto')
  kmeans.fit(lazdef_mlm_PA)
  LazDEF_pred = kmeans.labels_
  centroids = kmeans.cluster_centers_
  performance_list.append(print_metrics(LazDEF_sample_labels, LazDEF_pred))
print_avg_performance(performance_list)

LazDEF Vanilla-ESM Embeddings
Avgerage Accuracy: 0.6116720000000001 +- 6.40000000000085e-05
Avgerage Precision: 0.6015056549983335 +- 0.022316141051103133
Avgerage Recall: 0.506416 +- 0.000315569326773064
Avgerage F1 Score: 0.5495882163167652 +- 0.00501346505060804
Avgerage Auroc: 0.611672 +- 6.400000000002931e-05


In [ ]:
#@title High-N condition correct hp
tr_size=40000

# 1
model_list = [SVC(C=0.1, kernel='linear'), MLPClassifier(activation='tanh', hidden_layer_sizes=750), LogisticRegression(C=5), RandomForestClassifier(criterion='entropy', n_estimators=500), AdaBoostClassifier(learning_rate=1, n_estimators=500), KNeighborsClassifier(n_neighbors=50)]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif', ]

embs = lazbf_mlm_PA
labels = LazBF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazbf_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

# 2
model_list = [SVC(C=5), MLPClassifier(hidden_layer_sizes=50), LogisticRegression(C=1), RandomForestClassifier(criterion='log_loss', n_estimators=500), AdaBoostClassifier(learning_rate=1, n_estimators=500), KNeighborsClassifier(n_neighbors=50)]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif', ]

embs = lazdef_mlm_PA
labels = LazDEF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazdef_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

---
SVC performance on lazbf_mlm_none
Avgerage Accuracy: 0.90222 +- 0.0026858145877926994
Avgerage Precision: 0.8944691041967378 +- 0.0047687944960443165
Avgerage Recall: 0.9120799999999999 +- 0.0021637005338077525
Avgerage F1 Score: 0.9031799317669822 +- 0.002450323793546827
Avgerage Auroc: 0.90222 +- 0.0026858145877927016
---
MLP performance on lazbf_mlm_none
Avgerage Accuracy: 0.90626 +- 0.00279757037444994
Avgerage Precision: 0.9017221957877256 +- 0.0033785357673233613
Avgerage Recall: 0.91192 +- 0.0037791004220581346
Avgerage F1 Score: 0.9067868965294021 +- 0.0028001122038611146
Avgerage Auroc: 0.90626 +- 0.0027975703744499703
---
Logistic regression performance on lazbf_mlm_none
Avgerage Accuracy: 0.9023399999999999 +- 0.000926498785752064
Avgerage Precision: 0.8957435808662965 +- 0.0016686567570894872
Avgerage Recall: 0.9106800000000002 +- 0.0017554486605993326
Avgerage F1 Score: 0.9031477114203584 +- 0.000910020697766276
Avgerage Auroc: 0.9023399999999999 +- 0.00092649878575205

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Avgerage Accuracy: 0.8395399999999998 +- 0.0016752313273097482
Avgerage Precision: 0.8307658254123582 +- 0.00520510693120346
Avgerage Recall: 0.8529199999999999 +- 0.007979573923462328
Avgerage F1 Score: 0.8416492936595255 +- 0.0020559069033318327
Avgerage Auroc: 0.8395400000000001 +- 0.0016752313273097341
---
Logistic regression performance on lazdef_mlm_none
Avgerage Accuracy: 0.82668 +- 0.0043969989765748115
Avgerage Precision: 0.8208041165335626 +- 0.004118307359162984
Avgerage Recall: 0.8358399999999999 +- 0.005624802218745135
Avgerage F1 Score: 0.828249074753179 +- 0.004490919850331365
Avgerage Auroc: 0.82668 +- 0.004396998976574825
---
RF performance on lazdef_mlm_none
Avgerage Accuracy: 0.8211 +- 0.0005176871642217815
Avgerage Precision: 0.8143785867267861 +- 0.001247673961656645
Avgerage Recall: 0.8318 +- 0.003025227264190252
Avgerage F1 Score: 0.8229917639209681 +- 0.0009128649854364335
Avgerage Auroc: 0.8211 +- 0.0005176871642217815
---
AdaBoost performance on lazdef_mlm_non

In [ ]:
#@title Medium-N condition correct HP
tr_size=1000

# 1
model_list = [SVC(C=1, kernel='linear'), MLPClassifier(activation='tanh', hidden_layer_sizes=500), LogisticRegression(C=1), RandomForestClassifier(criterion='entropy', n_estimators=500), AdaBoostClassifier(learning_rate=0.1, n_estimators=200), KNeighborsClassifier(n_neighbors=10)]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif']

embs = lazbf_mlm_PA
labels = LazBF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazbf_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

# 2
model_list = [SVC(C=0.1, kernel='linear'), MLPClassifier(hidden_layer_sizes=1000), LogisticRegression(C=0.1), RandomForestClassifier(criterion='entropy', n_estimators=200), AdaBoostClassifier(learning_rate=1, n_estimators=500), KNeighborsClassifier(n_neighbors=25, weights='distance')]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif']

embs = lazdef_mlm_PA
labels = LazDEF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazdef_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

In [ ]:
#@title Low-N condition Correct HP
tr_size=100

# 1
model_list = [SVC(C=0.1, kernel='linear'), MLPClassifier(hidden_layer_sizes=750), LogisticRegression(C=1), RandomForestClassifier(criterion='entropy', n_estimators=50), AdaBoostClassifier(learning_rate=1), KNeighborsClassifier(n_neighbors=10)]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif']

embs = lazbf_mlm_PA
labels = LazBF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazbf_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

# 2
model_list = [SVC(C=1), MLPClassifier(hidden_layer_sizes=500), LogisticRegression(C=1), RandomForestClassifier(n_estimators=200), AdaBoostClassifier(learning_rate=1, n_estimators=200), KNeighborsClassifier(n_neighbors=25)]
model_names = ['SVC', 'MLP', 'Logistic regression', 'RF', 'AdaBoost', 'KNN classif']

embs = lazdef_mlm_PA
labels = LazDEF_sample_labels

for model_type, name in zip(model_list, model_names):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    print("---")
    print(f'{name} performance on lazdef_mlm_pa')
    performances = []
    for train_index, test_index in kf.split(embs, labels):
      X_train, X_test = embs[train_index][:tr_size], embs[test_index]
      y_train, y_test = labels[train_index][:tr_size], labels[test_index]

      steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model_type)
      ]
      pipeline = Pipeline(steps)
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      integer_predictions = [1 if p >= 0.5 else 0 for p in y_pred]

      performances.append(print_metrics(y_test, integer_predictions))
    print_avg_performance(performances)

---
SVC performance on lazbf_mlm_none
Avgerage Accuracy: 0.8291000000000001 +- 0.021114639471229445
Avgerage Precision: 0.8370055304535319 +- 0.03234480016958341
Avgerage Recall: 0.81944 +- 0.03242009253533989
Avgerage F1 Score: 0.8274184620616515 +- 0.021268113637958306
Avgerage Auroc: 0.8291000000000001 +- 0.021114639471229428
---
MLP performance on lazbf_mlm_none
Avgerage Accuracy: 0.8407 +- 0.012079072812099426
Avgerage Precision: 0.8267370152719629 +- 0.018192366725071584
Avgerage Recall: 0.8626799999999999 +- 0.006670052473556701
Avgerage F1 Score: 0.8442200247802528 +- 0.0103830074174369
Avgerage Auroc: 0.8407 +- 0.012079072812099422
---
Logistic regression performance on lazbf_mlm_none
Avgerage Accuracy: 0.8416600000000001 +- 0.011322473228054004
Avgerage Precision: 0.8397652153546604 +- 0.027997017136370213
Avgerage Recall: 0.8468 +- 0.02791100141521261
Avgerage F1 Score: 0.842470654488741 +- 0.010063143940911337
Avgerage Auroc: 0.8416600000000001 +- 0.011322473228054016
---
R